In [231]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt


# TP 6

In [232]:
def sepLine(line):
    return line.split(";")[1:]
def getArray(file):
    arr = []
    for line in file:
        line = sepLine(line)
        Line = []
        for x in line:

            x = x.replace(",",".")

            try:
                Line.append(float(x))
            except:
                Line.append(x)
        arr.append(Line)
    return arr


f = "Exo1.csv"
file = open(f,"r")

array = getArray(file)
array = array[1:]

In [233]:
class Knn:
    def __init__(self,data):
        self.data = data

    def manhattan(self,index=None,data=None):
        if data:
            element = data
        else:
            element = self.data[index]
        distances = {}
        for i in range(len(self.data)):
            somme=0
            for j in range(len(self.data[i]) - 1):
                somme += abs(element[j]-self.data[i][j])
            distances[i]=somme
        return distances
    
    def euclidienne(self,index=None,data=None):
        if data:
            element = data
        else:
            element = self.data[index]

        distances = {}
        for i in range(len(self.data)):
            somme=0
            for j in range(len(self.data[i]) - 1):
                somme += pow(element[j]-self.data[i][j],2)
            distances[i]=sqrt(somme)
        return distances
    
    def minkowski(self,index=None,p=1,data=None):
        if data:
            element = data
        else:
            element = self.data[index]

        distances = {}
        for i in range(len(self.data)):
            somme=0
            for j in range(len(self.data[i])-1):
                somme += pow(abs(element[j]-self.data[i][j]),p)
            distances[i]=pow(somme,1/p)
        return distances 
    
    def cosine(self,index=None,data=None):
        if data:
            element = data
        else:
            element = self.data[index]

        distances = {}
        for i in range(len(self.data)):
            somme=0
            ab = 0
            a2 = 0
            b2 = 0
            for j in range(len(self.data[i])-1):
                ab += self.data[i][j] * element[j]
                a2 += pow(self.data[i][j],2)
                b2 += pow(element[j],2)

            somme = ab/(sqrt(a2)*sqrt(b2))
            distances[i]= 1 - somme
        return distances
        pass
    
    def hamming(self,index=None,data=None):
        if data:
            element = data
        else:
            element = self.data[index]

        distances = {}
        for i in range(len(self.data)):
            somme=0
            for j in range(len(self.data[i]) - 1):
                if element[j] != self.data[i][j]:
                    somme+=1
            distances[i]=somme
    
        return distances

    def getK(self,distances,k):
        return sorted(distances.items(), key=lambda t: t[1])[:k]
    
    def getClass(self,elem,algo="euclidienne",k=3):
        classes = {}

        if algo == "euclidienne":
            distances = self.euclidienne(data=elem)
        if algo == "manhattan":
            distances = self.manhattan(data=elem)
        if algo == "minkowski":
            distances = self.minkowski(data=elem)
        if algo == "cosine":
            distances = self.cosine(data=elem)
        if algo == "hamming":
            distances = self.hamming(data=elem)
        
        distances = self.getK(distances,k)
        
        for (i,dist) in distances:

            if self.data[i][-1] in classes:
                classes[self.data[i][-1]] += 1
            else:
                classes[self.data[i][-1]] = 1

        return classes

In [234]:
import random


class kmeans:
    def __init__(self,pdata,K):
        self.K = K
        self.pdata = pdata
        self.centroids = []
        self.datas = {}
        for i in range(self.K):
            self.datas[i] = []

    def init_centroids(self):
        tier = int(self.pdata.shape[0]/3)
        for i in range(self.K):
            self.centroids.append(list(self.pdata.iloc[i*tier]))

    def init_data(self):
        for i in range(self.pdata.shape[0]):
            row = list(self.pdata.iloc[i])
            self.datas[self.getCluster(row)].append(row)

    def centroid(self,data):
        return data.mean()
    
    def updateCentroid(self):
        self.centroids = []
        for _,data in self.datas.items():
            df = pd.DataFrame(data)
            center = list(self.centroid(df))
            center[-1] = round(center[-1])
            self.centroids.append(center)

    def addToDatas(self,item):
        self.datas[self.getCluster(item)].append(item)

    def getCluster(self,item):
        self.Knn = Knn(self.centroids)
        center = self.Knn.getClass(item,algo="manhattan",k=1)
        return list(center.keys())[0]
    
    def updateDatas(self):
        for key in self.datas.keys():
            for elem in self.datas[key]:
                newCluster = self.getCluster(elem)
                if newCluster != key:
                    self.datas[key].remove(elem)
                    self.datas[newCluster].append(elem)
    
    def run(self,iter=10):
        self.init_centroids()
        self.init_data()

        for _ in range(iter):
            self.updateCentroid()
            self.updateDatas()
    

In [239]:
Data = pd.read_excel("Exo1.xlsx")
Data = Data.drop(columns=Data.columns[0])
items = Data["Class"].unique()

for i,txt in enumerate(items):
    print(i,txt)
    Data["Class"] = Data["Class"].replace(txt,i)

0 Iris-setosa
1 Iris-versicolor
2 Iris-virginica


In [236]:
kms = kmeans(pdata=Data,K=3)
kms.run()

In [238]:
kms.centroids

[[5.006, 3.418, 1.464, 0.244, 0],
 [5.904761904761905,
  2.7460317460317465,
  4.412698412698413,
  1.4333333333333333,
  1],
 [6.87027027027027,
  3.0864864864864856,
  5.745945945945946,
  2.089189189189189,
  2]]